<a href="https://colab.research.google.com/github/douglasmmachado/MedicineConsumption/blob/main/notebooks/unified_approach/5_Forecasting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 5 - Forecasting and prediction



---



---



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns
import math as m

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

from sklearn.tree import export_graphviz
from subprocess import call
from IPython.display import Image

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,  mean_absolute_percentage_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import StandardScaler

from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV



df_3_clusters_url ="https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/unified_approach/clustered/df_clustered_3.csv"
df_4_clusters_url ="https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/unified_approach/clustered/df_clustered_4.csv"
df_5_clusters_url ="https://raw.githubusercontent.com/douglasmmachado/MedicineConsumption/main/datasets/unified_approach/clustered/df_clustered_5.csv"
df_url = "https://raw.githubusercontent.com/douglasmmachado/ExploratoryDataAnalysis/main/datasets/unified_approach/df_ma.csv"

df = pd.read_csv(df_url)
df_3_clusters = pd.read_csv(df_3_clusters_url)
df_4_clusters = pd.read_csv(df_4_clusters_url)
df_5_clusters = pd.read_csv(df_5_clusters_url)

medicines = [3400892088310,3400892075761,3400892203645,
             3400892065366,3400892052120,3400891996128,
             3400893826706,3400893736135,3400893875490,
             3400890837149,3400891235203,3400891225037,
             3400891191226,3400892729589,3400892745848,
             3400892697789,3400892761527,3400893022634,
             3400892761695,3400892669236,3400892508566]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4074 entries, 0 to 4073
Data columns (total 48 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   HOSPI_CODE_UCD     4074 non-null   int64  
 1   LIT_HC             4074 non-null   float64
 2   LIT_HP             4074 non-null   float64
 3   N_ETB              4074 non-null   float64
 4   N_UFS              4074 non-null   float64
 5   PN_MEDICAL         4074 non-null   float64
 6   POPULATION         4074 non-null   float64
 7   P_MEDICAL          4074 non-null   float64
 8   QUANTITY           4074 non-null   float64
 9   QUANTITY_MA        4074 non-null   float64
 10  SEJ_HAD            4074 non-null   float64
 11  SEJ_MCO            4074 non-null   float64
 12  SEJ_PSY            4074 non-null   float64
 13  SEJ_SLD            4074 non-null   float64
 14  SEJ_SSR            4074 non-null   float64
 15  MONTH_1.0          4074 non-null   int64  
 16  MONTH_2.0          4074 

## 5.1 - New database composition based on clusters

## 5.2 - Building forecasting models based on clusters

In [ ]:
def test_1_baseline(df, medicine, df_scores):

  X = df[df['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY', 'HOSPI_CODE_UCD'], axis=1).values

  scaler = StandardScaler()
  X_scaled = scaler.fit_transform(X)

  y = df[df['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].values

  X_scaled, y = shuffle(X_scaled, y, random_state=42)

  # Split the data into training and testing sets
  X_train, X_test, y_train, y_test = train_test_split(X_scaled, y,
                                                      test_size = .2,
                                                      random_state = 42)
  print(f'Size of data set: {len(X)}')
  print(f'Size of training set: {len(X_train)}')
  print(f'Size of test set: {len(X_test)}')

  # Define the parameter distributions for RandomizedSearchCV
  param_grid = {
      'max_depth': np.arange(2, 20, 1),
      'n_estimators': np.arange(2, int(round(len(X_train)*0.1,0)), 1)
  }
  depth_len = param_grid['max_depth'].size
  estimators_len = param_grid['n_estimators'].size

  print(f'Size of grid search: {depth_len * estimators_len}')

  # Create the RandomizedSearchCV object
  grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                             param_grid=param_grid,
                             scoring = 'r2',
                             cv = 5,
                             n_jobs = -1,
                             verbose = 3)

  ''' >3 : the fold and candidate parameter indexes
      are also displayed together with the starting time of the computation.
  '''
  # Fit the RandomizedSearchCV object to the data
  grid_search.fit(X_train, y_train)

  # Get the best estimator
  best_estimator = grid_search.best_estimator_

  # Make predictions using the best estimator
  y_pred = best_estimator.predict(X_test)

  # Calculate R^2 score
  r2 = r2_score(y_test, y_pred)

  # Calculate MAE
  mae = mean_absolute_error(y_test, y_pred)

  # Calculate MAPE
  mape = mean_absolute_percentage_error(y_test, y_pred)

  # Calculate RMSE
  rmse = np.sqrt(mean_squared_error(y_test, y_pred))

  # Print the best parameters, best score, and evaluation metrics
  print('Medicine:' + str(medicine))
  print(f'Datapoints in test: {len(X_test)}')
  print('Best Parameters:', grid_search.best_params_)
  print('Training Score: R^2', round(grid_search.best_score_, 3))
  print('Test score: ')
  print('R^2 Score:', round(r2, 3))
  print('MAE:', round(mae, 3))
  print('MAPE:', round(mape, 3))
  print('RMSE:', round(rmse, 3))
  print()


  # Create the new row as a DataFrame
  new_row = pd.DataFrame({'HOSPI_CODE_UCD': ['CODE_UCD_'+str(medicine)],
                          'R2': [r2],
                          'RMSE': [rmse],
                          'MAE': [mae],
                          'MAPE': [mape]})

  # Append the new row to the DataFrame
  df_scores = pd.concat([df_scores, new_row], ignore_index=True)

  # Return the updated DataFrame
  return df_scores


In [ ]:
features = ['HOSPI_CODE_UCD', 'LIT_HC', 'LIT_HP', 'N_ETB', 'N_UFS', 'PN_MEDICAL',
       'POPULATION', 'P_MEDICAL', 'QUANTITY', 'QUANTITY_MA', 'SEJ_HAD',
       'SEJ_MCO', 'SEJ_PSY', 'SEJ_SLD', 'SEJ_SSR', 'MONTH_1.0', 'MONTH_2.0',
       'MONTH_3.0', 'MONTH_4.0', 'MONTH_5.0', 'MONTH_6.0', 'MONTH_7.0',
       'MONTH_8.0', 'MONTH_9.0', 'MONTH_10.0', 'MONTH_11.0', 'MONTH_12.0']

In [ ]:
df_prediction_scores = pd.DataFrame(columns=['HOSPI_CODE_UCD', 'R2', 'RMSE', 'MAE', 'MAPE'])

for medicine in medicines:

  df_prediction_scores = test_1_baseline(df[features], medicine, df_prediction_scores)

df_prediction_scores

Size of data set: 171
Size of training set: 136
Size of test set: 35
Size of grid search: 216
Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Medicine:3400892088310
Datapoints in test: 35
Best Parameters: {'max_depth': 7, 'n_estimators': 3}
Training Score: R^2 0.814
Test score: 
R^2 Score: 0.775
MAE: 607.882
MAPE: 8.533
RMSE: 860.776

Size of data set: 204
Size of training set: 163
Size of test set: 41
Size of grid search: 252
Fitting 5 folds for each of 252 candidates, totalling 1260 fits
Medicine:3400892075761
Datapoints in test: 41
Best Parameters: {'max_depth': 7, 'n_estimators': 15}
Training Score: R^2 0.667
Test score: 
R^2 Score: 0.385
MAE: 371.478
MAPE: 0.149
RMSE: 590.036

Size of data set: 200
Size of training set: 160
Size of test set: 40
Size of grid search: 252
Fitting 5 folds for each of 252 candidates, totalling 1260 fits
Medicine:3400892203645
Datapoints in test: 40
Best Parameters: {'max_depth': 5, 'n_estimators': 14}
Training Score: R^2 0.863
Test scor

,HOSPI_CODE_UCD,R2,RMSE,MAE,MAPE
0,CODE_UCD_3400892088310,0.775266,860.776201,607.882137,8.533207
1,CODE_UCD_3400892075761,0.384943,590.036073,371.477666,0.149066
2,CODE_UCD_3400892203645,0.936251,875.124638,569.658408,0.686666
3,CODE_UCD_3400892065366,0.830707,769.829387,504.420878,0.325603
4,CODE_UCD_3400892052120,0.808439,370.278037,211.330776,0.354099
5,CODE_UCD_3400891996128,0.794586,15856.203777,8209.707325,1.041047
6,CODE_UCD_3400893826706,0.856255,954.863299,655.912796,1.986171
7,CODE_UCD_3400893736135,0.895540,713.645111,557.441821,0.332676
8,CODE_UCD_3400893875490,0.725888,2563.055639,1525.765244,5.094943
9,CODE_UCD_3400890837149,0.834535,2915.524717,1519.411433,13.672224


In [2]:
def test_2_clustering(df, df_scores, medicines):
  for cluster in df.CLUSTER.unique():

    X = df[df['CLUSTER'] == cluster].drop(['QUANTITY', 'QUANTITY_MA', 'CLUSTER'], axis=1).copy().values
    y = df[df['CLUSTER'] == cluster]['QUANTITY'].copy().values

    X, y = shuffle(X, y, random_state = 42)

    # Perform the train-test split with shuffled samples
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                      test_size = .2,
                                                      random_state = 42)
    print(f'Size of data set: {len(X)}')
    print(f'Size of training set: {len(X_train)}')
    print(f'Size of test set: {len(X_test)}')

    df_test = pd.DataFrame(X_test, columns = df.drop(['QUANTITY', 'QUANTITY_MA', 'CLUSTER'], axis=1).copy().columns)
    df_test['QUANTITY'] = y_test

    # Define the parameter distributions for RandomizedSearchCV
    param_grid = {
        'max_depth': np.arange(2, 20, 1),
        'n_estimators': np.arange(2, max(int(m.ceil(len(X_train)*0.1)),2), 1)
    }
    depth_len = param_grid['max_depth'].size
    estimators_len = param_grid['n_estimators'].size

    print(f'Size of grid search: {depth_len * estimators_len}')

    # Create the RandomizedSearchCV object
    grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                              param_grid=param_grid,
                              scoring = 'r2',
                              cv = 5,
                              n_jobs = -1,
                              verbose = 3)

    ''' >3 : the fold and candidate parameter indexes
        are also displayed together with the starting time of the computation.
    '''

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)

    # Fit the RandomizedSearchCV object to the data
    grid_search.fit(X_train_scaled, y_train)
    print("Finished training")

    # Get the best estimator
    best_estimator = grid_search.best_estimator_

    for medicine in df_test.HOSPI_CODE_UCD.unique():

      X_test_medicine = df_test[df_test['HOSPI_CODE_UCD'] == medicine].drop(['QUANTITY'], axis=1).copy().values

      scaler = StandardScaler()
      X_test_scaled = scaler.fit_transform(X_test_medicine)

      y_test_medicine = df_test[df_test['HOSPI_CODE_UCD'] == medicine]['QUANTITY'].copy().values

      # Make predictions using the best estimator
      y_pred = best_estimator.predict(X_test_scaled)

      # Calculate R^2 score
      r2 = r2_score(y_test_medicine, y_pred)

      # Calculate MAE
      mae = mean_absolute_error(y_test_medicine, y_pred)

      # Calculate MAPE
      mape = mean_absolute_percentage_error(y_test_medicine, y_pred)

      # Calculate RMSE
      rmse = np.sqrt(mean_squared_error(y_test_medicine, y_pred))

      # Print the best parameters, best score, and evaluation metrics
      print('Medicine:' + str(medicine))
      print(f'Datapoints in test: {len(X_test)}')
      print('Best Parameters:', grid_search.best_params_)
      print('Training Score: R^2', round(grid_search.best_score_, 3))
      print('Test score: ')
      print('R^2 Score:', round(r2, 3))
      print('MAE:', round(mae, 3))
      print('MAPE:', round(mape, 3))
      print('RMSE:', round(rmse, 3))
      print()


      # Create the new row as a DataFrame
      new_row = pd.DataFrame({'HOSPI_CODE_UCD': ['CODE_UCD_'+str(int(medicine))],
                              'CLUSTER': [cluster],
                              'R2': [r2],
                              'RMSE': [rmse],
                              'MAE': [mae],
                              'MAPE': [mape]})

      # Append the new row to the DataFrame
      df_scores = pd.concat([df_scores, new_row], ignore_index=True)

  # Return the updated DataFrame
  return df_scores

In [ ]:
df_prediction_scores_3_clusters = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_3_clusters = test_2_clustering(df_3_clusters, df_prediction_scores_3_clusters, medicines)

df_prediction_scores_3_clusters

Size of data set: 1906
Size of training set: 1524
Size of test set: 382
Size of grid search: 2718
Fitting 5 folds for each of 2718 candidates, totalling 13590 fits
Finished training
Medicine:3400892065366.0
Datapoints in test: 382
Best Parameters: {'max_depth': 18, 'n_estimators': 99}
Training Score: R^2 0.92
Test score: 
R^2 Score: -0.677
MAE: 2588.66
MAPE: 0.527
RMSE: 3211.088

Medicine:3400891225037.0
Datapoints in test: 382
Best Parameters: {'max_depth': 18, 'n_estimators': 99}
Training Score: R^2 0.92
Test score: 
R^2 Score: -1.338
MAE: 12673.158
MAPE: 1.045
RMSE: 14641.399

Medicine:3400892761527.0
Datapoints in test: 382
Best Parameters: {'max_depth': 18, 'n_estimators': 99}
Training Score: R^2 0.92
Test score: 
R^2 Score: 0.184
MAE: 3711.731
MAPE: 0.814
RMSE: 4228.341

Medicine:3400892729589.0
Datapoints in test: 382
Best Parameters: {'max_depth': 18, 'n_estimators': 99}
Training Score: R^2 0.92
Test score: 
R^2 Score: -6.962
MAE: 2524.942
MAPE: 0.552
RMSE: 3053.691

Medicine:3

,HOSPI_CODE_UCD,CLUSTER,R2,RMSE,MAE,MAPE
0,CODE_UCD_3400892065366,0,-0.677230,3211.088120,2588.659936,0.527428
1,CODE_UCD_3400891225037,0,-1.337908,14641.399258,12673.157890,1.044777
2,CODE_UCD_3400892761527,0,0.183941,4228.341482,3711.731483,0.814089
3,CODE_UCD_3400892729589,0,-6.962100,3053.690820,2524.941980,0.552157
4,CODE_UCD_3400893875490,0,-0.688895,3901.238562,3284.471038,0.557338
...,...,...,...,...,...,...
58,CODE_UCD_3400892203645,2,-5.559959,3639.081931,3368.384307,0.294737
59,CODE_UCD_3400891996128,2,-9.916208,116585.842242,111147.202774,0.933478
60,CODE_UCD_3400891225037,2,-80.907671,16451.617178,16324.159159,0.674006
61,CODE_UCD_3400893022634,2,-3.150775,4027.249553,3398.831713,1.329645


In [3]:
df_prediction_scores_4_clusters = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_4_clusters = test_2_clustering(df_4_clusters, df_prediction_scores_4_clusters, medicines)

df_prediction_scores_4_clusters

Size of data set: 1208
Size of training set: 966
Size of test set: 242
Size of grid search: 1710
Fitting 5 folds for each of 1710 candidates, totalling 8550 fits
Finished training
Medicine:3400891996128.0
Datapoints in test: 242
Best Parameters: {'max_depth': 15, 'n_estimators': 18}
Training Score: R^2 0.909
Test score: 
R^2 Score: -3.121
MAE: 42730.085
MAPE: 0.938
RMSE: 48382.901

Medicine:3400893875490.0
Datapoints in test: 242
Best Parameters: {'max_depth': 15, 'n_estimators': 18}
Training Score: R^2 0.909
Test score: 
R^2 Score: -78.86
MAE: 6331.067
MAPE: 0.697
RMSE: 6790.945

Medicine:3400892052120.0
Datapoints in test: 242
Best Parameters: {'max_depth': 15, 'n_estimators': 18}
Training Score: R^2 0.909
Test score: 
R^2 Score: -1.35
MAE: 1169.622
MAPE: 1.055
RMSE: 1755.25

Medicine:3400890837149.0
Datapoints in test: 242
Best Parameters: {'max_depth': 15, 'n_estimators': 18}
Training Score: R^2 0.909
Test score: 
R^2 Score: 0.407
MAE: 2180.04
MAPE: 1.104
RMSE: 3287.05

Medicine:34

,HOSPI_CODE_UCD,CLUSTER,R2,RMSE,MAE,MAPE
0,CODE_UCD_3400891996128,0,-3.121134,48382.901109,42730.084986,0.937660
1,CODE_UCD_3400893875490,0,-78.859926,6790.944848,6331.067227,0.697159
2,CODE_UCD_3400892052120,0,-1.349822,1755.250192,1169.621746,1.054625
3,CODE_UCD_3400890837149,0,0.407016,3287.049805,2180.039750,1.104136
4,CODE_UCD_3400893826706,0,-0.982477,1835.350206,1353.112197,9.444871
...,...,...,...,...,...,...
79,CODE_UCD_3400891225037,3,-5.342061,17488.733493,16320.765867,0.798786
80,CODE_UCD_3400891235203,3,-367.580066,5736.018994,4946.714839,3.533744
81,CODE_UCD_3400892508566,3,-23198.307644,3489.933597,3488.369876,50.860356
82,CODE_UCD_3400893736135,3,-18.739847,2686.135193,2293.910053,0.469879


In [5]:
df_prediction_scores_5_clusters = pd.DataFrame(columns=[ 'HOSPI_CODE_UCD', 'CLUSTER', 'R2', 'RMSE', 'MAE', 'MAPE'])

df_prediction_scores_5_clusters = test_2_clustering(df_5_clusters, df_prediction_scores_5_clusters, medicines)

df_prediction_scores_5_clusters

Size of data set: 522
Size of training set: 417
Size of test set: 105
Size of grid search: 720
Fitting 5 folds for each of 720 candidates, totalling 3600 fits
Finished training
Medicine:3400892761695.0
Datapoints in test: 105
Best Parameters: {'max_depth': 14, 'n_estimators': 35}
Training Score: R^2 0.702
Test score: 
R^2 Score: -0.136
MAE: 614.247
MAPE: 0.589
RMSE: 914.746

Medicine:3400893826706.0
Datapoints in test: 105
Best Parameters: {'max_depth': 14, 'n_estimators': 35}
Training Score: R^2 0.702
Test score: 
R^2 Score: 0.617
MAE: 586.772
MAPE: 2.084
RMSE: 831.442

Medicine:3400893022634.0
Datapoints in test: 105
Best Parameters: {'max_depth': 14, 'n_estimators': 35}
Training Score: R^2 0.702
Test score: 
R^2 Score: 0.612
MAE: 419.247
MAPE: 10.861
RMSE: 535.121

Medicine:3400892075761.0
Datapoints in test: 105
Best Parameters: {'max_depth': 14, 'n_estimators': 35}
Training Score: R^2 0.702
Test score: 
R^2 Score: -0.02
MAE: 880.229
MAPE: 0.761
RMSE: 1105.04

Medicine:340089373613

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Medicine:3400892508566.0
Datapoints in test: 105
Best Parameters: {'max_depth': 14, 'n_estimators': 35}
Training Score: R^2 0.702
Test score: 
R^2 Score: 0.652
MAE: 640.437
MAPE: 0.308
RMSE: 641.977

Size of data set: 683
Size of training set: 546
Size of test set: 137
Size of grid search: 954
Fitting 5 folds for each of 954 candidates, totalling 4770 fits
Finished training
Medicine:3400893736135.0
Datapoints in test: 137
Best Parameters: {'max_depth': 9, 'n_estimators': 49}
Training Score: R^2 0.879
Test score: 
R^2 Score: -198.214
MAE: 2833.987
MAPE: 1.212
RMSE: 4126.47

Medicine:3400892745848.0
Datapoints in test: 137
Best Parameters: {'max_depth': 9, 'n_estimators': 49}
Training Score: R^2 0.879
Test score: 
R^2 Score: -17.874
MAE: 4234.194
MAPE: 0.579
RMSE: 4536.937

Medicine:3400892761695.0
Datapoints in test: 137
Best Parameters: {'max_depth': 9, 'n_estimators': 49}
Training Score: R^2 0.879
Test score: 
R^2 Score: -55.096
MAE: 2115.459
MAPE: 1.468
RMSE: 2134.57

Medicine:340089

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:918: UndefinedMetricWarning: R^2 score is not well-defined with less than two samples.
  warnings.warn(msg, UndefinedMetricWarning)


Training Score: R^2 0.879
Test score: 
R^2 Score: -88.823
MAE: 2623.624
MAPE: 0.665
RMSE: 3415.875

Medicine:3400891235203.0
Datapoints in test: 137
Best Parameters: {'max_depth': 9, 'n_estimators': 49}
Training Score: R^2 0.879
Test score: 
R^2 Score: -1.075
MAE: 1453.222
MAPE: 24.791
RMSE: 2530.67

Medicine:3400892065366.0
Datapoints in test: 137
Best Parameters: {'max_depth': 9, 'n_estimators': 49}
Training Score: R^2 0.879
Test score: 
R^2 Score: -55.135
MAE: 2034.766
MAPE: 0.319
RMSE: 2444.969

Medicine:3400892669236.0
Datapoints in test: 137
Best Parameters: {'max_depth': 9, 'n_estimators': 49}
Training Score: R^2 0.879
Test score: 
R^2 Score: -1587.783
MAE: 7411.541
MAPE: 0.67
RMSE: 7413.875

Size of data set: 1398
Size of training set: 1118
Size of test set: 280
Size of grid search: 1980
Fitting 5 folds for each of 1980 candidates, totalling 9900 fits
Finished training
Medicine:3400892697789.0
Datapoints in test: 280
Best Parameters: {'max_depth': 11, 'n_estimators': 7}
Trainin

,HOSPI_CODE_UCD,CLUSTER,R2,RMSE,MAE,MAPE
0,CODE_UCD_3400892761695,4,-0.136058,914.745797,614.247277,0.589399
1,CODE_UCD_3400893826706,4,0.617021,831.442165,586.771871,2.083996
2,CODE_UCD_3400893022634,4,0.611710,535.120760,419.247164,10.861079
3,CODE_UCD_3400892075761,4,-0.020473,1105.039680,880.229313,0.760727
4,CODE_UCD_3400893736135,4,0.800840,419.625169,306.015569,0.235413
...,...,...,...,...,...,...
99,CODE_UCD_3400892203645,2,-5.559959,3639.081931,3368.384307,0.294737
100,CODE_UCD_3400891996128,2,-9.916208,116585.842242,111147.202774,0.933478
101,CODE_UCD_3400891225037,2,-80.907671,16451.617178,16324.159159,0.674006
102,CODE_UCD_3400893022634,2,-3.150775,4027.249553,3398.831713,1.329645
